In [23]:
import os
import plaid
import requests
import datetime
import json
import pandas as pd
%matplotlib inline

In [24]:
def pretty_print_response(response):
  print(json.dumps(response, indent=4, sort_keys=True))

In [25]:
PLAID_CLIENT_ID = os.getenv('PLAID_CLIENT_ID')
PLAID_SECRET = os.getenv('PLAID_SECRET')
PLAID_PUBLIC_KEY = os.getenv('PLAID_PUBLIC_KEY')
PLAID_ENV = os.getenv('PLAID_ENV', 'sandbox')
PLAID_PRODUCTS = os.getenv('PLAID_PRODUCTS', 'transactions')

# Plaid Access Token

In this section, you will use the plaid-python api to generate the correct authentication tokens to access data in the free developer Sandbox. This mimics how you might connect to your own account or a customer account, but due to privacy issues, this homework will only require connecting to and analyzing the fake data from the developer sandbox that Plaid provides. 

Complete the following steps to generate an access token:
1. Create a client to connect to plaid
2. Use the client to generate a public token and request the following items: 
['transactions', 'income', 'assets']
3. Exchange the public token for an access token
4. Test the access token by requesting and printing the available test accounts

### 1. Create a client to connect to plaid

In [26]:
INSTITUTION_ID = "ins_109508"

In [27]:
# YOUR CODE HERE
client = plaid.Client(
    client_id='5e8f6f5f547cd5001384720e', 
    secret='e20cf6f31e3fe615b475a45e8e5589', 
    public_key='7016bfcf84fec71e1e6a80651637c6', 
    environment='sandbox')

### 2. Generate a public token

In [28]:
# YOUR CODE HERE
# Select an institution for processing
INSTITUTION_ID = "ins_109512"

# Create public token to be exchanged for institution access token
create_tkn_response = client.Sandbox.public_token.create(INSTITUTION_ID, ['transactions','income','assets'])
create_tkn_response

{'public_token': 'public-sandbox-9763c5ec-4374-4f16-9f8d-ba3ba5934c02',
 'request_id': '4NfRdrJk9tXvjOW'}

### 3. Exchange the public token for an access token

In [29]:
# YOUR CODE HERE
# Exchange public token for access token
exchange_response = client.Item.public_token.exchange(create_tkn_response['public_token'])
exchange_response

{'access_token': 'access-sandbox-a90b444e-3039-43b9-ad78-89c8690a1048',
 'item_id': 'yX9gV3mq6phKWWnpnLRPSdqqyg8m1KtyGrEjA',
 'request_id': 'TbTE0hIQ5Nqq4tU'}

### 4. Fetch Accounts

In [32]:
# YOUR CODE HERE
# Store access token as variable
access_token = exchange_response['access_token']
# Get accounts associated with institution
client.Accounts.get(access_token)

{'accounts': [{'account_id': 'Mz9x7wWqnmU8kk3p3d4qSzQQw3oorAc9gxBAj',
   'balances': {'available': 100,
    'current': 110,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '0000',
   'name': 'Plaid Checking',
   'official_name': 'Plaid Gold Standard 0% Interest Checking',
   'subtype': 'checking',
   'type': 'depository'},
  {'account_id': '1qrxlyXZAahxllLQL3MpHxRRX4nnL3f5d7gyA',
   'balances': {'available': 200,
    'current': 210,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '1111',
   'name': 'Plaid Saving',
   'official_name': 'Plaid Silver Standard 0.1% Interest Saving',
   'subtype': 'savings',
   'type': 'depository'},
  {'account_id': 'LrG3VjElA9h7ZZBLBaNlI1llk7BB5rhPn8b4Z',
   'balances': {'available': None,
    'current': 1000,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '2222',
   'name': 'Plaid CD',
   'officia

---

# Account Transactions with Plaid

In this section, you will use the Plaid Python SDK to connect to the Developer Sandbox account and grab a list of transactions. You will need to complete the following steps:


1. Use the access token to fetch the transactions for the last 90 days
2. Print the categories for each transaction type
3. Create a new DataFrame using the following fields from the JSON transaction data: `date, name, amount, category`. (For categories with more than one label, just use the first category label in the list)
4. Convert the data types to the appropriate types (i.e. datetimeindex for the date and float for the amount)

### 1. Fetch the Transactions for the last 90 days

In [38]:
# YOUR CODE HERE
start_date = '{:%Y-%m-%d}'.format(datetime.datetime.now() + datetime.timedelta(-90))
end_date = '{:%Y-%m-%d}'.format(datetime.datetime.now())

### 2. Print the categories for each transaction

In [39]:
# YOUR CODE HERE
transaction_response = client.Transactions.get(access_token,start_date,end_date)

### 3. Create a new DataFrame using the following fields from the JSON transaction data: date, name, amount, category. 

(For categories with more than one label, just use the first category label in the list)

In [35]:
# YOUR CODE HERE
print(json.dumps(transaction_response['transactions'][:2],indent=4, sort_keys=True))

[
    {
        "account_id": "eG4rvaNERzcEnnJoJ6MzImV9ennn6eSLZDNK6",
        "account_owner": null,
        "amount": 500,
        "authorized_date": null,
        "category": [
            "Travel",
            "Airlines and Aviation Services"
        ],
        "category_id": "22001000",
        "date": "2020-03-31",
        "iso_currency_code": "USD",
        "location": {
            "address": null,
            "city": null,
            "country": null,
            "lat": null,
            "lon": null,
            "postal_code": null,
            "region": null,
            "store_number": null
        },
        "name": "United Airlines",
        "payment_channel": "in store",
        "payment_meta": {
            "by_order_of": null,
            "payee": null,
            "payer": null,
            "payment_method": null,
            "payment_processor": null,
            "ppd_id": null,
            "reason": null,
            "reference_number": null
        },
        "pendi

### 4. Convert the data types to the appropriate types 

(i.e. datetimeindex for the date and float for the amount)

In [36]:
# YOUR CODE HERE

---

# Income Analysis with Plaid

In this section, you will use the Plaid Sandbox to complete the following:
1. Determine the previous year's gross income and print the results
2. Determine the current monthly income and print the results
3. Determine the projected yearly income and print the results

In [37]:
# YOUR CODE HERE
previous_year_income=
current_month_income=
projected_yearly_income=

SyntaxError: invalid syntax (<ipython-input-37-728b3eec2d66>, line 2)

---

# Budget Analysis
In this section, you will use the transactions DataFrame to analyze the customer's budget

1. Calculate the total spending per category and print the results (Hint: groupby or count transactions per category)
2. Generate a bar chart with the number of transactions for each category 
3. Calculate the expenses per month
4. Plot the total expenses per month

### Calculate the expenses per category

In [ ]:
# YOUR CODE HERE

### Calculate the expenses per month

In [ ]:
# YOUR CODE HERE